### Imports

In [1]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
# from concurrent.futures import ThreadPoolExecutor
from tqdm.contrib.concurrent import thread_map
pd.set_option('display.max_colwidth', None)

### GET AIRLINES URLS ###

In [2]:
url = 'https://www.pilotjobsnetwork.com/'
max_urls = 5

def get_airlines_urls(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    links = soup.find_all('a')
    for link in links:
        href = link.get('href')
        if href and 'jobs/' in href:
            airline_url = url + href
            airline_name = link.text.strip()
            yield airline_url, airline_name # Generator creation

### GET INFO FROM PAGES ###

In [3]:
def get_airline_tables(airline_tuple):
    airline_url, airline_name = airline_tuple
    try:
        list_tables = pd.read_html(airline_url)
        salary = list_tables[3].iloc[2:4,1].tolist()
        # iloc iloc[2:4,2] gets you the dates, but remove the salary ... smthing to 
        last_update = list_tables[3].iloc[2:4,2].tolist()
        return [airline_url, airline_name] + salary + last_update
    except Exception as e:
        print(e)
        try:
            return list_tables[3]
        except NameError:
            return airline_url

### MAP IT !! ###

In [4]:
list_salaries = list(thread_map(get_airline_tables, get_airlines_urls(url)))
# print dans un excel le list_salaries

0it [00:00, ?it/s]

'ascii' codec can't encode characters in position 18-19: ordinal not in range(128)
URL can't contain control characters. '/jobs/Hello\r\n\r\nI_recently_visited_your_site__ppjn.com__from_a_Google_search_using_the_Google_chrome_browser._\r\n\r\nI_wanted_to_reach_out_as_I_noticed_it_displayed_a_not_secure_warning_just_to_the_left_of_where_the_url_is_displayed,_rather_than_the_usual_padlock_icon_you_see_on_similar_sites_to_yours._Do_you_get_that_as_well_when_you_visit_your_site?_If_you_read_on_Google' (found at least '\r')


### DF AS OUTPUT ###

In [5]:
df_salary = pd.DataFrame([salary for salary in list_salaries if type(salary) == list])
df_salary

,0,1,2,3,4,5
0,https://www.pilotjobsnetwork.com/jobs/DAS_Private_Jets_GmbH,13Mar - Germany (Fr) - DAS Private Jets GmbH,NaN,NaN,NaN,NaN
1,https://www.pilotjobsnetwork.com/jobs/Air_One,13Mar - UK (Ca) - Air One,NaN,NaN,NaN,NaN
2,https://www.pilotjobsnetwork.com/jobs/Air_Belgium,13Mar - Belgium (Ch) - Air Belgium,7500Ã¢âÂ¬ before taxes to be paid in Belgium,NaN,23/Oct/17,NaN
3,https://www.pilotjobsnetwork.com/jobs/Air_Atlanta_Icelandic,13Mar - Iceland (Ch) - Air Atlanta Icelandic,Daily fee of 600 USD + 50 USD of per diem and overtime of 39% of your daily per flight hour above 65h,Daily fee of 400 USD + per diem of 50USD. Overtime of 39% of daily fee per extra hour over 65h,10/Jul/22,10/Jul/22
4,https://www.pilotjobsnetwork.com/jobs/Aerologic,12Mar - Germany (Ca) - Aerologic,"207.936,72",13731672,9/Nov/22,9/Nov/22
...,...,...,...,...,...,...
494,https://www.pilotjobsnetwork.com/jobs/Titan_Airways_Ltd,22Aug - UK (Re) - Titan Airways Ltd,100420,93420,1/Oct/18,1/Oct/18
495,https://www.pilotjobsnetwork.com/jobs/Aeropartner,18Aug - Czech Republic (Fr) - Aeropartner,NaN,2200Ã¢âÂ¬ gross,NaN,18/Aug/19
496,https://www.pilotjobsnetwork.com/jobs/Airnorth,13Aug - Australia (Re) - Airnorth,NaN,"45000 bras, 40 metro",NaN,2/Jun/08
497,https://www.pilotjobsnetwork.com/jobs/FlyinGroup,13Aug - Belgium (Fr) - FlyinGroup,NaN,NaN,NaN,NaN


### ERRORS LIST ###

In [6]:
[salary for salary in list_salaries if type(salary) == str]

['https://www.pilotjobsnetwork.com/jobs/Air_CaraÃ¯bes_Atlantique',
 'https://www.pilotjobsnetwork.com/jobs/Hello\r\n\r\nI_recently_visited_your_site__ppjn.com__from_a_Google_search_using_the_Google_chrome_browser._\r\n\r\nI_wanted_to_reach_out_as_I_noticed_it_displayed_a_not_secure_warning_just_to_the_left_of_where_the_url_is_displayed,_rather_than_the_usual_padlock_icon_you_see_on_similar_sites_to_yours._Do_you_get_that_as_well_when_you_visit_your_site?_If_you_read_on_Google']

### DF REWORK ###

In [7]:
# First name columns
df_salary.columns = ["URL", "AirlineName", "CaptMax", "CaptMin", "DateCaptMax", "DateCaptMin"]
df_salary

,URL,AirlineName,CaptMax,CaptMin,DateCaptMax,DateCaptMin
0,https://www.pilotjobsnetwork.com/jobs/DAS_Private_Jets_GmbH,13Mar - Germany (Fr) - DAS Private Jets GmbH,NaN,NaN,NaN,NaN
1,https://www.pilotjobsnetwork.com/jobs/Air_One,13Mar - UK (Ca) - Air One,NaN,NaN,NaN,NaN
2,https://www.pilotjobsnetwork.com/jobs/Air_Belgium,13Mar - Belgium (Ch) - Air Belgium,7500Ã¢âÂ¬ before taxes to be paid in Belgium,NaN,23/Oct/17,NaN
3,https://www.pilotjobsnetwork.com/jobs/Air_Atlanta_Icelandic,13Mar - Iceland (Ch) - Air Atlanta Icelandic,Daily fee of 600 USD + 50 USD of per diem and overtime of 39% of your daily per flight hour above 65h,Daily fee of 400 USD + per diem of 50USD. Overtime of 39% of daily fee per extra hour over 65h,10/Jul/22,10/Jul/22
4,https://www.pilotjobsnetwork.com/jobs/Aerologic,12Mar - Germany (Ca) - Aerologic,"207.936,72",13731672,9/Nov/22,9/Nov/22
...,...,...,...,...,...,...
494,https://www.pilotjobsnetwork.com/jobs/Titan_Airways_Ltd,22Aug - UK (Re) - Titan Airways Ltd,100420,93420,1/Oct/18,1/Oct/18
495,https://www.pilotjobsnetwork.com/jobs/Aeropartner,18Aug - Czech Republic (Fr) - Aeropartner,NaN,2200Ã¢âÂ¬ gross,NaN,18/Aug/19
496,https://www.pilotjobsnetwork.com/jobs/Airnorth,13Aug - Australia (Re) - Airnorth,NaN,"45000 bras, 40 metro",NaN,2/Jun/08
497,https://www.pilotjobsnetwork.com/jobs/FlyinGroup,13Aug - Belgium (Fr) - FlyinGroup,NaN,NaN,NaN,NaN


### PIVOT TABLE

In [8]:
df_stack = (df_salary.set_index(['URL', 'AirlineName', 'DateCaptMax', 'DateCaptMin'])
   .rename_axis(['Top/Base'], axis=1)
   .stack(dropna=False) # Put True to remove NaNs
   .reset_index())
df_stack.columns = ['URL', 'AirlineName', 'DateCaptMax', 'DateCaptMin', 'Top/Base', 'Salary']
df_stack

,URL,AirlineName,DateCaptMax,DateCaptMin,Top/Base,Salary
0,https://www.pilotjobsnetwork.com/jobs/DAS_Private_Jets_GmbH,13Mar - Germany (Fr) - DAS Private Jets GmbH,NaN,NaN,CaptMax,NaN
1,https://www.pilotjobsnetwork.com/jobs/DAS_Private_Jets_GmbH,13Mar - Germany (Fr) - DAS Private Jets GmbH,NaN,NaN,CaptMin,NaN
2,https://www.pilotjobsnetwork.com/jobs/Air_One,13Mar - UK (Ca) - Air One,NaN,NaN,CaptMax,NaN
3,https://www.pilotjobsnetwork.com/jobs/Air_One,13Mar - UK (Ca) - Air One,NaN,NaN,CaptMin,NaN
4,https://www.pilotjobsnetwork.com/jobs/Air_Belgium,13Mar - Belgium (Ch) - Air Belgium,23/Oct/17,NaN,CaptMax,7500Ã¢âÂ¬ before taxes to be paid in Belgium
...,...,...,...,...,...,...
993,https://www.pilotjobsnetwork.com/jobs/Airnorth,13Aug - Australia (Re) - Airnorth,NaN,2/Jun/08,CaptMin,"45000 bras, 40 metro"
994,https://www.pilotjobsnetwork.com/jobs/FlyinGroup,13Aug - Belgium (Fr) - FlyinGroup,NaN,NaN,CaptMax,NaN
995,https://www.pilotjobsnetwork.com/jobs/FlyinGroup,13Aug - Belgium (Fr) - FlyinGroup,NaN,NaN,CaptMin,NaN
996,https://www.pilotjobsnetwork.com/jobs/MS_AVIATION,12Aug - Austria (Fr) - MS AVIATION,NaN,NaN,CaptMax,NaN


### Select date (min/max) based on "CaptMin/CaptMax"

In [9]:
# Create a unique column for date depending on Top/Base column value. Extract Year
df_stack['Date_reworked'] = np.where(df_stack['Top/Base'] == 'CaptMax', df_stack['DateCaptMax'], df_stack['DateCaptMin'])
df_stack['Date_reworked'] = pd.to_datetime(df_stack['Date_reworked'])
df_stack['Year'] = df_stack['Date_reworked'].dt.year
df_stack['Year'] = df_stack['Year'].astype('Int64')


# Split column AirlineName on " - " seperator
df_stack[["DateFromLink", "Country", "Name"]] = df_stack["AirlineName"].str.split(" - ", expand=True)

# # Cleaning columns. Droping non necessary
clean_order = ['URL', 'Country', 'Name', 'Year', 'Top/Base', 'Salary']
df_stack = df_stack[clean_order]
# df_stack.to_excel('airline_list.xlsx')


# Json creation

df_stack.to_json('airline_scraping_algolia.json')
df_stack

,URL,Country,Name,Year,Top/Base,Salary
0,https://www.pilotjobsnetwork.com/jobs/DAS_Private_Jets_GmbH,Germany (Fr),DAS Private Jets GmbH,<NA>,CaptMax,NaN
1,https://www.pilotjobsnetwork.com/jobs/DAS_Private_Jets_GmbH,Germany (Fr),DAS Private Jets GmbH,<NA>,CaptMin,NaN
2,https://www.pilotjobsnetwork.com/jobs/Air_One,UK (Ca),Air One,<NA>,CaptMax,NaN
3,https://www.pilotjobsnetwork.com/jobs/Air_One,UK (Ca),Air One,<NA>,CaptMin,NaN
4,https://www.pilotjobsnetwork.com/jobs/Air_Belgium,Belgium (Ch),Air Belgium,2017,CaptMax,7500Ã¢âÂ¬ before taxes to be paid in Belgium
...,...,...,...,...,...,...
993,https://www.pilotjobsnetwork.com/jobs/Airnorth,Australia (Re),Airnorth,2008,CaptMin,"45000 bras, 40 metro"
994,https://www.pilotjobsnetwork.com/jobs/FlyinGroup,Belgium (Fr),FlyinGroup,<NA>,CaptMax,NaN
995,https://www.pilotjobsnetwork.com/jobs/FlyinGroup,Belgium (Fr),FlyinGroup,<NA>,CaptMin,NaN
996,https://www.pilotjobsnetwork.com/jobs/MS_AVIATION,Austria (Fr),MS AVIATION,<NA>,CaptMax,NaN


### MAKING THE JSON ###

In [10]:
df_stack.to_json("airline_scraping_algolia.json", orient="records")